<a href="https://colab.research.google.com/github/rifatozkurt/hdl/blob/master/586_tutorial_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it. \\

Then you need to install Git-LFS. Uncomment the following instructions.

In [ ]:
%%capture
!pip install datasets transformers
!apt install git-lfs

In [ ]:
import huggingface_hub
from huggingface_hub import notebook_login
your_token = 'my_token' #Please generate a token from huggingface
huggingface_hub.login(token = your_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/). We will focus on CoLA dataset.


The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a  dataset containing sentences labeled grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for each one of those tasks and use the `Trainer` API to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [ ]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [ ]:
task = "cola"
model_checkpoint = "bert-base-uncased"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
from datasets import load_dataset, load_metric

Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions. `load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [ ]:
actual_task = "mnli" if task == "mnli-mm" else task
print(f"Task is: {actual_task} ")
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Task is: cola 


Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-6-883afb0b9b04>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', actual_task)


The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [ ]:
metric



NameError: ignored

To access an actual element, you need to select a split first, then give an index:

In [ ]:
dataset["train"][0]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

,sentence,label,idx
0,I didn't help him because I have any sympathy for urban guerillas.,acceptable,5242
1,"Pat has become crazy, and Chris an incredible bore.",acceptable,7112
2,"The most you want, the least you eat.",unacceptable,103
3,Which student were you talking about?,acceptable,3929
4,Myself shaved me.,unacceptable,7893
5,John saw more horses than Bill saw cows or Pete talked to.,unacceptable,6559
6,My eyes are itching my brother.,unacceptable,3160
7,it convinced Bill that Mary should sleep.,unacceptable,7670
8,The boys fought with each other.,acceptable,7519
9,Who do you think that would be nominated for the position?,unacceptable,4784


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [ ]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'matthews_correlation': 0.058823529411764705}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can double check it does work on our current dataset. Recall the dataset object again:

In [ ]:
dataset['train'][0]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0}

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: Our friends won't buy this analysis, let alone the next one we propose.


We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(dataset['train'][:5])


{'input_ids': [[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 1998, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 2030, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 1996, 2062, 2057, 2817, 16025, 1010, 1996, 13675, 16103, 2121, 2027, 2131, 1012, 102], [101, 2154, 2011, 2154, 1996, 8866, 2024, 2893, 14163, 8024, 3771, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels):

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1063
    })
})

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, BertForSequenceClassification
import torch
from transformers import logging

logging.set_verbosity_error() #Since we learn the meaning of warning, we can silence it.

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2

model_checkpoint = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model.dropout.p = 0.2


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name. Otherwise, it will be pushed directly to your own account.

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/bert-base-uncased-finetuned-cola is already a clone of https://huggingface.co/rifatozkurt/bert-base-uncased-finetuned-cola. Make sure you pull the latest changes with `repo.git_pull()`.


You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [ ]:
# Recall that we have CUDA device. Lets check the amount of memory.
!nvidia-smi

Sat May  6 12:07:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   78C    P0    34W /  70W |   2947MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'loss': 0.5019, 'learning_rate': 1.308411214953271e-06, 'epoch': 0.93}
{'eval_loss': 0.45924490690231323, 'eval_matthews_correlation': 0.5153179547856604, 'eval_runtime': 1.3184, 'eval_samples_per_second': 791.118, 'eval_steps_per_second': 50.061, 'epoch': 1.0}
{'train_runtime': 88.543, 'train_samples_per_second': 96.575, 'train_steps_per_second': 6.042, 'train_loss': 0.49436726614693616, 'epoch': 1.0}


TrainOutput(global_step=535, training_loss=0.49436726614693616, metrics={'train_runtime': 88.543, 'train_samples_per_second': 96.575, 'train_steps_per_second': 6.042, 'train_loss': 0.49436726614693616, 'epoch': 1.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [ ]:
trainer.evaluate()

{'eval_loss': 0.45924490690231323, 'eval_matthews_correlation': 0.5153179547856604, 'eval_runtime': 1.4294, 'eval_samples_per_second': 729.697, 'eval_steps_per_second': 46.174, 'epoch': 1.0}


{'eval_loss': 0.45924490690231323,
 'eval_matthews_correlation': 0.5153179547856604,
 'eval_runtime': 1.4294,
 'eval_samples_per_second': 729.697,
 'eval_steps_per_second': 46.174,
 'epoch': 1.0}

To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
%%capture
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
To https://huggingface.co/rifatozkurt/bert-base-uncased-finetuned-cola
   c159b43..3204895  main -> main

   c159b43..3204895  main -> main

To https://huggingface.co/rifatozkurt/bert-base-uncased-finetuned-cola
   3204895..75bc3a1  main -> main

   3204895..75bc3a1  main -> main



Now you can always download your model with `AutoModelForSequenceClassification` with the model name.

In [ ]:
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("rifatozkurt/bert-base-uncased-finetuned-cola")
#check your own model also

## Hyperparameter search

The `Trainer` supports hyperparameter search using [optuna](https://optuna.org/) or [Ray Tune](https://docs.ray.io/en/latest/tune/). For this last section you will need either of those libraries installed, just uncomment the line you want on the next cell and run it. I prefer Optuna.

In [ ]:
%%capture
! pip install optuna
# ! pip install ray[tune]

During hyperparameter search, the `Trainer` will run several trainings, so it needs to have the model defined via a function (so it can be reinitialized at each new run) instead of just having it passed. We just use the same function as before:

In [ ]:
def model_init():
    return BertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

And we can instantiate our `Trainer` like before:

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/bert-base-uncased-finetuned-cola is already a clone of https://huggingface.co/rifatozkurt/bert-base-uncased-finetuned-cola. Make sure you pull the latest changes with `repo.git_pull()`.


`Trainer` has a function `hyperparameter_search` which takes `hp_space` function as input. It has typing hint as: (Callable[ [Trial] , Union[float, Sequence[float]] ]). Please check [documentation](https://huggingface.co/docs/transformers/hpo_train) for further details. I suggest to not use default `hyperparameter_search`. You might not able to tune all hyperparameters easily. You can follow the second approach that I mention. Source code of that approach is available in Moodle.

In [ ]:
import optuna

def my_hp_space(trial: optuna.trial.Trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-5, log=True),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [1,2,3]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
    }

In [ ]:
best_run = trainer.hyperparameter_search(hp_space = my_hp_space, backend = "optuna", n_trials=2, direction="maximize")

[I 2023-05-06 13:00:21,299] A new study created in memory with name: no-name-8ecf963d-ce0e-4046-84c3-d8274b27a034
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'eval_loss': 0.6069502234458923, 'eval_matthews_correlation': 0.0, 'eval_runtime': 1.3436, 'eval_samples_per_second': 776.272, 'eval_steps_per_second': 49.122, 'epoch': 1.0}


[I 2023-05-06 13:01:39,380] Trial 0 finished with value: 0.0 and parameters: {'learning_rate': 2.0308813630951254e-06, 'num_train_epochs': 1, 'per_device_train_batch_size': 64}. Best is trial 0 with value: 0.0.


{'train_runtime': 76.418, 'train_samples_per_second': 111.898, 'train_steps_per_second': 1.754, 'train_loss': 0.6176583589012943, 'epoch': 1.0}


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'loss': 0.5122, 'learning_rate': 5.715971606636895e-06, 'epoch': 0.93}
{'eval_loss': 0.4803115725517273, 'eval_matthews_correlation': 0.4895445706047777, 'eval_runtime': 1.3687, 'eval_samples_per_second': 762.062, 'eval_steps_per_second': 48.223, 'epoch': 1.0}
{'loss': 0.3629, 'learning_rate': 3.129559114945992e-06, 'epoch': 1.87}
{'eval_loss': 0.44341841340065, 'eval_matthews_correlation': 0.5805514135255713, 'eval_runtime': 1.3389, 'eval_samples_per_second': 779.011, 'eval_steps_per_second': 49.295, 'epoch': 2.0}
{'loss': 0.2857, 'learning_rate': 5.431466232550896e-07, 'epoch': 2.8}
{'eval_loss': 0.5283095240592957, 'eval_matthews_correlation': 0.5704260999182836, 'eval_runtime': 1.3434, 'eval_samples_per_second': 776.395, 'eval_steps_per_second': 49.129, 'epoch': 3.0}


[I 2023-05-06 13:06:11,017] Trial 1 finished with value: 0.5704260999182836 and parameters: {'learning_rate': 8.302384098327798e-06, 'num_train_epochs': 3, 'per_device_train_batch_size': 16}. Best is trial 1 with value: 0.5704260999182836.


{'train_runtime': 269.9835, 'train_samples_per_second': 95.017, 'train_steps_per_second': 5.945, 'train_loss': 0.37967854199751144, 'epoch': 3.0}


The `hyperparameter_search` method returns a `BestRun` objects, which contains the value of the objective maximized (by default the sum of all metrics) and the hyperparameters it used for that run.

In [ ]:
best_run

BestRun(run_id='1', objective=0.5704260999182836, hyperparameters={'learning_rate': 8.302384098327798e-06, 'num_train_epochs': 3, 'per_device_train_batch_size': 16}, run_summary=None)

You can customize the objective to maximize by passing along a `compute_objective` function to the `hyperparameter_search` method, and you can customize the search space by passing a `hp_space` argument to `hyperparameter_search`. See this [forum post](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/10) for some examples.

To reproduce the best training, just set the hyperparameters in your `TrainingArgument` before creating a `Trainer`:

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'loss': 0.5122, 'learning_rate': 5.715971606636895e-06, 'epoch': 0.93}
{'eval_loss': 0.4803115725517273, 'eval_matthews_correlation': 0.4895445706047777, 'eval_runtime': 1.4584, 'eval_samples_per_second': 715.146, 'eval_steps_per_second': 45.254, 'epoch': 1.0}
{'loss': 0.3629, 'learning_rate': 3.129559114945992e-06, 'epoch': 1.87}
{'eval_loss': 0.44341841340065, 'eval_matthews_correlation': 0.5805514135255713, 'eval_runtime': 1.3709, 'eval_samples_per_second': 760.82, 'eval_steps_per_second': 48.144, 'epoch': 2.0}
{'loss': 0.2857, 'learning_rate': 5.431466232550896e-07, 'epoch': 2.8}
{'eval_loss': 0.5283095240592957, 'eval_matthews_correlation': 0.5704260999182836, 'eval_runtime': 1.3469, 'eval_samples_per_second': 774.377, 'eval_steps_per_second': 49.002, 'epoch': 3.0}
{'train_runtime': 264.304, 'train_samples_per_second': 97.059, 'train_steps_per_second': 6.073, 'train_loss': 0.37967854199751144, 'epoch': 3.0}


TrainOutput(global_step=1605, training_loss=0.37967854199751144, metrics={'train_runtime': 264.304, 'train_samples_per_second': 97.059, 'train_steps_per_second': 6.073, 'train_loss': 0.37967854199751144, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.44341841340065, 'eval_matthews_correlation': 0.5805514135255713, 'eval_runtime': 1.3891, 'eval_samples_per_second': 750.839, 'eval_steps_per_second': 47.512, 'epoch': 3.0}


{'eval_loss': 0.44341841340065,
 'eval_matthews_correlation': 0.5805514135255713,
 'eval_runtime': 1.3891,
 'eval_samples_per_second': 750.839,
 'eval_steps_per_second': 47.512,
 'epoch': 3.0}

In [ ]:
trainer.push_to_hub()

Upload file runs/May06_12-25-39_2358607b6c26/events.out.tfevents.1683378403.2358607b6c26.162.26:   0%|        …

Upload file runs/May06_12-25-39_2358607b6c26/events.out.tfevents.1683378674.2358607b6c26.162.28:   0%|        …

To https://huggingface.co/rifatozkurt/bert-base-uncased-finetuned-cola
   0b3077b..9870502  main -> main

   0b3077b..9870502  main -> main

To https://huggingface.co/rifatozkurt/bert-base-uncased-finetuned-cola
   9870502..c64bbb1  main -> main

   9870502..c64bbb1  main -> main



'https://huggingface.co/rifatozkurt/bert-base-uncased-finetuned-cola/commit/987050256e9d40e6d453b629e00e055f92c188c2'

Now we can evaluate the best trained model on test set

I want to list some additional resources that you might want to check.


*   [Simple Training with the Huggingface Transformers Trainer](https://www.youtube.com/watch?v=u--UVvH-LIQ&t=174s&ab_channel=HuggingFace)
*   

